In [1]:
from tqdm import tqdm
from datetime import datetime
from modules.utils import *
import os

## Input job search keyword

In [2]:
keyword = 'data analyst'
pages = 0

# with open('config.yaml', 'r') as f:
#     config = yaml.safe_load(f)
#     keyword_list = config['job_titles']
#     pages = config['pages']

In [3]:
# links = get_job_links(keyword, start_page=0, pages=pages)
links, main_soup = get_job_links_selenium(keyword, pages=pages, debug=True)

In [ ]:
main = {}
soups = {}
for index, link in tqdm(enumerate(links), total = len(links), dynamic_ncols =True):
    try:
        main[index], soups[index] = get_job_info(link, index, return_soup=True)
    except Exception as e:
        logger.error(e, exc_info=True)

In [21]:
# links
# len(main)
# soups[3]

NoneType

In [4]:
df = process_df(main, remove_nulls=True, remove_duplicates=True)
print(len(df))
df.head(3)

31


,date_logged,company,job_title,level,job_type,experience,spark,degree,descriptions,industry1,industry2,link
job_id,,,,,,,,,,,,
3793581487,2024-01-05,BAH Partners,Data Analyst,Associate,Full-time,•My client is seeking a skilled and motivated ...,,•Bachelor's or higher degree in a quantitative...,\nMy client is seeking a skilled and motivated...,Engineering and Finance,Financial Services and Investment Management,https://sg.linkedin.com/jobs/view/data-analyst...
3764392364,2024-01-05,Capgemini,Data Analyst,Entry level,Full-time,•We are looking for a skilled Data Analyst who...,•We are looking for a skilled Data Analyst who...,"•Bachelor's degree in a relevant field (e.g., ...",\nAbout Capgemini\n\nCapgemini is a global lea...,"Information Technology, Engineering, and Other","IT Services and IT Consulting, Software Develo...",https://sg.linkedin.com/jobs/view/data-analyst...
3775345393,2024-01-05,Dyson,Lead Data Analyst,Mid-Senior level,Full-time,,,,\nAbout Us\nWe are currently seeking a Lead Da...,Information Technology,"Appliances, Electrical, and Electronics Manufa...",https://sg.linkedin.com/jobs/view/lead-data-an...


### Save the file

In [5]:
most_recent_file = f"BATCH_{datetime.now().strftime('%Y-%m-%d-%H%M')}.xlsx"
df.to_excel(f"ignore/{most_recent_file}", engine='xlsxwriter')
most_recent_filepaths = [file for file in os.listdir('ignore') if datetime.now().strftime('%Y-%m-%d') in file]
most_recent_filepaths

['BATCH_2024-01-05-1413.xlsx', 'MAIN_2024-01-05.xlsx']

## Append all BATCH files to MAIN

In [1]:
from modules.utils import *

batchfiles = [file for file in os.listdir('ignore') if file.startswith('BATCH')]
batchfiles = ['ignore/' + filename for filename in batchfiles]
batchfiles

['ignore/BATCH_2024-01-05-1413.xlsx',
 'ignore/BATCH_2024-01-11.xlsx',
 'ignore/BATCH_2024-01-18.xlsx']

In [2]:
file_directory = sorted([file for file in os.listdir('ignore') if file.startswith('MAIN')])
if len(file_directory)>0:
    mainfile = file_directory[-1] # Gets latest file
    mainfilepath = f'ignore/{mainfile}'
mainfilepath

'ignore/MAIN_2024-01-11.xlsx'

In [3]:
# main_df = update_main('files/MAIN_2023-12-25.xlsx', most_recent_filepaths)
# main_df = update_main('files/MAIN_2023-12-27.xlsx', [df])
main_df = update_main(mainfilepath, batchfiles)

In [4]:
main_df.to_excel(f"ignore/MAIN_{datetime.now().strftime('%Y-%m-%d')}.xlsx", engine='xlsxwriter')